In [74]:
import pandas as pd
import sqlite3 as sq
import numpy as np


Just imported libraries

In [75]:
stud_name = np.array([], dtype=object)
stud_age = np.array([], dtype=int)
stud_score = np.array([], dtype=int)
stud_passed = np.array([], dtype=bool)

In [76]:
data = pd.DataFrame({'Name': stud_name, 'Age': stud_age, 'Score': stud_score, 'Passed': stud_passed},
                    index=range(len(stud_name)),dtype = object)

In [82]:
def add_student():
    global stud_name, stud_age, stud_score, stud_passed
    stud_name = np.append(stud_name, input('Enter name: '))
    stud_age = np.append(stud_age, input('Enter age: '))
    x = int(input('Enter score: '))
    stud_score = np.append(stud_score, x)
    if x >= 60:
        stud_passed = np.append(stud_passed, True)
    else:
        stud_passed = np.append(stud_passed, False)
    with sq.connect('students.db') as con:
        df = pd.DataFrame({'name': stud_name, 'age': stud_age, 'score': stud_score, 'passed': stud_passed}, dtype=object)
        df.to_sql('students', con, if_exists='replace', index=False)

Function to add a student to the database

In [78]:
def show_students():
    with sq.connect('students.db') as con:
        df = pd.read_sql('SELECT * FROM students', con)
        df['passed'] = df['passed'].astype(bool)
        display(df)

Function to show all students

In [88]:
def clear_database():
    global stud_name, stud_age, stud_score, stud_passed
    stud_name = np.array([], dtype=object)
    stud_age = np.array([], dtype=int)
    stud_score = np.array([], dtype=int)
    stud_passed = np.array([], dtype=bool)
    with sq.connect('students.db') as con:
        con.execute('DELETE FROM students')
        con.commit()
    print("All data has been cleared!")

Function to clear the database

In [86]:
def score_mean():
    with sq.connect('students.db') as con:
        df = pd.read_sql('SELECT * FROM students', con)
        mean_score = df['score'].mean()
        print(f"Mean score: {mean_score:.2f}")

In [89]:
try:
    while True:
        choice = input("""Enter your choice: (1/2/3/4/5)

                            1. Add student
                            2. Show students
                            3. Clear database
                            4. Calculate mean score
                            5. Exit
                            """)
        match choice:
            case '1':
                add_student()
            case '2':
                show_students()
                break
            case '3':
                clear_database()
                break
            case '4':
                score_mean()
                break
            case '5':
                break
            case _:
                print("Invalid choice. Please try again.")

except ValueError:
    print("Invalid choice. Please try again.")
except KeyboardInterrupt:
    print("Program interrupted by user.")
except Exception as e:
    print(f"An error occurred: {e}")

Mean score: 79.67
